In [52]:
import torch

from pytorch_quantization.tensor_quant import QuantDescriptor
from pytorch_quantization.nn.modules.tensor_quantizer import TensorQuantizer

from pytorch_quantization import tensor_quant

In [53]:
quant_desc = QuantDescriptor(
    num_bits=5,
    fake_quant=True,
    # axis=(1),
    unsigned=False
    # amax=3.0
)

quantizer = TensorQuantizer(quant_desc)

In [54]:
d = torch.randn(4, 4)

quantizer.enable_calib()
quantizer.disable_quant()
quantizer(d)
quantizer.load_calib_amax()
quantizer.enable_quant()
quantizer.disable_calib()

W0815 10:38:30.598164 139950796811136 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W0815 10:38:30.602174 139950796811136 tensor_quantizer.py:173] Disable MaxCalibrator


In [55]:
print(d)
res = quantizer(d)
print(res)
print(type(res))
print(quantizer.extra_repr())
print(quantizer.amax)
print(quantizer.step_size)

tensor([[-1.3153,  0.1733,  1.3470, -0.5235],
        [ 0.9053, -1.4281, -0.5574, -0.7609],
        [ 0.5443, -1.0818,  1.2443, -0.5277],
        [ 0.6591,  1.6441,  1.9719, -0.2160]])
tensor([[-1.3146,  0.1315,  1.3146, -0.5258],
        [ 0.9202, -1.4461, -0.5258, -0.7888],
        [ 0.5258, -1.0517,  1.1832, -0.5258],
        [ 0.6573,  1.7090,  1.9719, -0.2629]])
<class 'torch.Tensor'>
5bit narrow fake per-tensor amax=1.9719 calibrator=MaxCalibrator scale=1.0 quant
tensor(1.9719)
tensor(0.1315)


In [56]:
import pytorch_quantization.nn as quant_nn
from torch import nn

In [63]:
w = torch.linspace(0, 10, 25).reshape( (5,5) )
b = torch.linspace(0, 5, 5)
d = torch.randn( (5, 5) )

fc = nn.Linear(5, 5, bias=True)

fc.weight = nn.Parameter(w)
fc.bias = nn.Parameter(b)

res = fc(d)
print(res)

tensor([[  0.4715,   1.1894,   1.9073,   2.6252,   3.3431],
        [  1.3406,   5.0019,   8.6632,  12.3244,  15.9857],
        [ -2.8714,  -6.6169, -10.3624, -14.1079, -17.8534],
        [ -1.6520,  -5.8589, -10.0657, -14.2726, -18.4795],
        [ -4.9389,  -8.3278, -11.7168, -15.1057, -18.4946]],
       grad_fn=<AddmmBackward0>)


In [83]:
input_desc = QuantDescriptor(num_bits=8, amax=30.0)

quant_fc = quant_nn.Linear(5, 5, bias=True, 
    quant_desc_input=input_desc,
    quant_desc_weight=tensor_quant.QUANT_DESC_8BIT_LINEAR_WEIGHT_PER_ROW)
quant_fc.weight = nn.Parameter(w)
quant_fc.bias = nn.Parameter(b)
res = quant_fc(d)
print(res)
input_desc,
quant_fc.extra_repr()

tensor([[  0.2976,   0.5525,   0.8182,   1.1142,   1.3358],
        [  1.2710,   4.9956,   8.7062,  12.3641,  16.1786],
        [ -2.6660,  -6.3388, -10.0101, -13.6992, -17.3014],
        [ -1.3826,  -5.0694,  -8.7406, -12.3886, -16.0552],
        [ -4.6221,  -7.3014,  -9.9884, -12.6831, -15.3856]],
       grad_fn=<AddmmBackward0>)


'in_features=5, out_features=5, bias=True'

In [84]:
print(d)
print(quant_fc._input_quantizer(d))
print(w)
print(quant_fc._weight_quantizer(w))
print(quant_fc._input_quantizer._get_amax(d))


tensor([[-0.8958,  0.7250, -0.1243, -0.4962,  0.5360],
        [ 0.8914, -0.6906, -0.4559,  0.8301,  0.5824],
        [ 1.0150, -1.3410, -0.9649, -0.8075, -0.2995],
        [-0.5101, -0.7255, -0.7554, -0.7846,  0.1563],
        [ 1.5592, -0.9597,  0.4358, -1.2826, -1.9794]])
tensor([[-0.9449,  0.7087, -0.2362, -0.4724,  0.4724],
        [ 0.9449, -0.7087, -0.4724,  0.9449,  0.4724],
        [ 0.9449, -1.4173, -0.9449, -0.7087, -0.2362],
        [-0.4724, -0.7087, -0.7087, -0.7087,  0.2362],
        [ 1.6535, -0.9449,  0.4724, -1.1811, -1.8898]])
tensor([[ 0.0000,  0.4167,  0.8333,  1.2500,  1.6667],
        [ 2.0833,  2.5000,  2.9167,  3.3333,  3.7500],
        [ 4.1667,  4.5833,  5.0000,  5.4167,  5.8333],
        [ 6.2500,  6.6667,  7.0833,  7.5000,  7.9167],
        [ 8.3333,  8.7500,  9.1667,  9.5833, 10.0000]])
tensor([[ 0.0000,  0.4199,  0.8399,  1.2467,  1.6667],
        [ 2.0965,  2.5098,  2.9232,  3.3366,  3.7500],
        [ 4.1798,  4.5932,  5.0066,  5.4199,  5.8333],
       